In [1]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from rdkit.Chem import AllChem, MolFromSmiles
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import DataStructs
from rdkit import Chem
from rxnfp.tokenization import get_default_tokenizer, SmilesTokenizer
from rdkit.Chem import rdChemReactions
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
def combnfeats_MLR_evaby_crossval_metric(feats, df_x, df_y, n_fold, model, metric, score):
    print('----------Linear regression of n features evaby crossval starts----------')
    """_Screen and select Well behalved features from of a list of 3 features_
    Evaluated by sum of 5-fold cross validation.
    This is because just taking consideration of one result from one MLR can cause lost of important features due to randomness.
    If the displaied R^2 is also high, there's a higher chance that these features are of high predictive value.
    If the displaied R^2 is 0/-/unexplainable, it can be due to accident, and further cross-validation can verify their importance.
    """
    # Create a list to store features with high, medium, or low score
    List_cvd_3 = [] 
    List_cvd_2 = [] 
    List_cvd_0 = [] 
    # for r2 and explained variance the closer to 1 the better
    if metric == 'r2' or metric == 'explained_variance':
        sum_score = [score*1,score*0.8,score*0.5]
    # for nmae, nrmse and max error, the closer to 0 the better, therefore we need to readjust the rank
    elif metric == 'neg_mean_absolute_error' or metric == 'neg_root_mean_squared_error' or metric == 'max_error':
        sum_score = [score/1,score/0.8,score/0.5]
    
    # Three different ranks for high, medium and low, based on the score we imputed
    rank_score = sorted(sum_score,reverse=True)
    score_1 = rank_score[0]
    score_2 = rank_score[1]
    score_3 = rank_score[2]
    
    for m in feats:
        fts = feats.index(m)
        df_repeated_cv_x = df_x
        df_repeated_cv_y = df_y
        sum_cvd = 0
        for k in range(0,n_fold):
            # Shuffle the data
            df_repeated_cv_x = df_repeated_cv_x.sample(frac=1,random_state=k)
            df_nf = df_repeated_cv_x[list(feats[fts])]
            df_repeated_cv_y = df_repeated_cv_y.sample(frac=1,random_state=k)
            cvd_scores = cross_val_score(model, df_nf, df_repeated_cv_y, cv=5,scoring= metric)
            sum_cvd += sum(cvd_scores)
        # Calculate average R^2
        avg_cvd = sum_cvd/n_fold/5
        if avg_cvd >= score_1: 
            print('Significant!')
            print(f'For features {feats[fts]} with {metric} = {avg_cvd}')
            List_cvd_3.append(feats[fts]) 
        elif avg_cvd >=score_2:
            List_cvd_2.append(feats[fts])
        elif avg_cvd >=score_3:
            List_cvd_0.append(feats[fts])
        #print(f'For features {feats[fts]} with {metric} = {avg_cvd}'+'cvd_scores = ',cvd_scores,sum_cvd,avg_cvd)   
    return List_cvd_3,List_cvd_2,List_cvd_0

In [3]:
# Data processing
df_origin = pd.read_excel('Afterscreen.xlsx', sheet_name='Result2')
df_origin = df_origin.sample(frac=1,random_state=42)
# Reindex the data
df_origin = df_origin.set_index('ID')


df_x = df_origin.drop(columns = ['Yield-avg','ligand','buch','Unnamed: 0','SMILES'])
df_y = df_origin[['Yield-avg']]
# standardize
scaler = StandardScaler()
df_x=pd.DataFrame(scaler.fit_transform(df_x),index=df_x.index, columns=df_x.columns)
df_std_origin = df_x
zero_std_cols = df_x.columns[df_x.std() == 0]
df_x=df_x[df_x.columns.difference(zero_std_cols)]
print (f"Dropping {len(zero_std_cols)} zero-variance features {zero_std_cols}")

Dropping 4 zero-variance features Index(['x119', 'x122', 'x144', 'x147'], dtype='object')


In [4]:
highest_ranked_combination_feats = [('x99', 'x23', 'x33'),
 ('x23', 'x33', 'x102'),
 ('x23', 'x33', 'x158'),
 ('x23', 'x33', 'x98'),
 ('x186', 'x23', 'x102'),
 ('x32', 'x23', 'x98'),
 ('x32', 'x23', 'x160'),
 ('x32', 'x99', 'x23'),
 ('x23', 'x6', 'x98'),
 ('x23', 'x33', 'x159'),
 ('x188', 'x33', 'x48'),
 ('x32', 'x23', 'x102'),
 ('x32', 'x23', 'x158'),
 ('x23', 'x175', 'x102'),
 ('x23', 'x6', 'x102'),
 ('x32', 'x23', 'x159'),
 ('x23', 'x67', 'x33')]

# Combination of four features

In [5]:
# Combination of four features
# for each combination in highest_ranked_combination_feats, use a for loop to add the 4th feature from df_origin.columns
extended_combinations = []

# iterate over the combinations in highest_ranked_combination_feats
for combination in highest_ranked_combination_feats:
    for column in df_x.columns:
        new_combination = combination + (column,)
        extended_combinations.append(new_combination)

In [6]:
# get rid of replicates
extended_combinations = list(set(extended_combinations))
print(len(extended_combinations))

model = LinearRegression()

3587


In [7]:
stop

NameError: name 'stop' is not defined

R2

In [ ]:
List2_cvd2_3_r2,List2_cvd2_2_r2,List2_cvd2_0_r2 = combnfeats_MLR_evaby_crossval_metric(extended_combinations,df_x,df_y,5,model,'r2',0.35)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x160') with r2 = 0.35848499306957676
Significant!
For features ('x32', 'x23', 'x98', 'x84') with r2 = 0.5609291055430733
Significant!
For features ('x23', 'x33', 'x158', 'x136') with r2 = 0.4189271089403438
Significant!
For features ('x23', 'x33', 'x159', 'x10') with r2 = 0.4029412572541283
Significant!
For features ('x23', 'x175', 'x102', 'x21') with r2 = 0.39022283480424014
Significant!
For features ('x32', 'x23', 'x158', 'x193') with r2 = 0.5487391583791401
Significant!
For features ('x23', 'x67', 'x33', 'x72') with r2 = 0.36247662099397987
Significant!
For features ('x23', 'x6', 'x98', 'x84') with r2 = 0.3716770666209982
Significant!
For features ('x186', 'x23', 'x102', 'x75') with r2 = 0.3514104566385245
Significant!
For features ('x23', 'x33', 'x102', 'x6') with r2 = 0.4094641844040573
Significant!
For features ('x99', 'x23', 'x33', 'x197') with r2 = 0.464032

In [ ]:
len(List2_cvd2_3_r2)

2935

In [ ]:
List3_cvd2_3_r2,List3_cvd2_2_r2,List3_cvd2_0_r2 = combnfeats_MLR_evaby_crossval_metric(List2_cvd2_3_r2,df_x,df_y,5,model,'r2',0.4)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x84') with r2 = 0.5609291055430733
Significant!
For features ('x23', 'x33', 'x158', 'x136') with r2 = 0.4189271089403438
Significant!
For features ('x23', 'x33', 'x159', 'x10') with r2 = 0.4029412572541283
Significant!
For features ('x32', 'x23', 'x158', 'x193') with r2 = 0.5487391583791401
Significant!
For features ('x23', 'x33', 'x102', 'x6') with r2 = 0.4094641844040573
Significant!
For features ('x99', 'x23', 'x33', 'x197') with r2 = 0.4640329490384465
Significant!
For features ('x32', 'x23', 'x102', 'x33') with r2 = 0.49722463522438015
Significant!
For features ('x32', 'x23', 'x158', 'x29') with r2 = 0.47967681673645535
Significant!
For features ('x23', 'x175', 'x102', 'x2') with r2 = 0.41252077975339646
Significant!
For features ('x32', 'x23', 'x102', 'x78') with r2 = 0.48751693026372145
Significant!
For features ('x99', 'x23', 'x33', 'x2') with r2 = 0.449710

In [ ]:
len(List3_cvd2_3_r2)

2199

In [ ]:
List4_cvd2_3_r2,List4_cvd2_2_r2,List4_cvd2_0_r2 = combnfeats_MLR_evaby_crossval_metric(List3_cvd2_3_r2,df_x,df_y,5,model,'r2',0.5)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x84') with r2 = 0.5609291055430733
Significant!
For features ('x32', 'x23', 'x158', 'x193') with r2 = 0.5487391583791401
Significant!
For features ('x32', 'x99', 'x23', 'x99') with r2 = 0.500797511147916
Significant!
For features ('x32', 'x23', 'x102', 'x127') with r2 = 0.5070269518181241
Significant!
For features ('x32', 'x23', 'x158', 'x106') with r2 = 0.5665015697841556
Significant!
For features ('x32', 'x23', 'x102', 'x213') with r2 = 0.5156463764338144
Significant!
For features ('x23', 'x33', 'x98', 'x25') with r2 = 0.5091813638572479
Significant!
For features ('x32', 'x23', 'x160', 'x141') with r2 = 0.5106345363903735
Significant!
For features ('x32', 'x23', 'x158', 'x64') with r2 = 0.512512960370296
Significant!
For features ('x32', 'x23', 'x158', 'x154') with r2 = 0.5398225242431507
Significant!
For features ('x23', 'x33', 'x159', 'x174') with r2 = 0.519901

In [ ]:
len(List4_cvd2_3_r2)

341

In [ ]:
List5_cvd2_3_r2,List5_cvd2_2_r2,List5_cvd2_0_r2 = combnfeats_MLR_evaby_crossval_metric(List4_cvd2_3_r2,df_x,df_y,10,model,'r2',0.53)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x158', 'x106') with r2 = 0.5344293496005046
Significant!
For features ('x32', 'x23', 'x98', 'x177') with r2 = 0.5368030985028025
Significant!
For features ('x32', 'x23', 'x158', 'x163') with r2 = 0.5312007795153455
Significant!
For features ('x32', 'x23', 'x98', 'x79') with r2 = 0.530204536062063
Significant!
For features ('x32', 'x23', 'x98', 'x83') with r2 = 0.5420285236941219
Significant!
For features ('x32', 'x23', 'x98', 'x77') with r2 = 0.5350648615917315
Significant!
For features ('x32', 'x23', 'x159', 'x183') with r2 = 0.5452833027091784
Significant!
For features ('x32', 'x23', 'x160', 'x140') with r2 = 0.5437921285336964
Significant!
For features ('x32', 'x99', 'x23', 'x183') with r2 = 0.5790135958067768
Significant!
For features ('x32', 'x99', 'x23', 'x177') with r2 = 0.531718346320866
Significant!
For features ('x32', 'x23', 'x98', 'x183') with r2 = 0.573098205

In [ ]:
len(List5_cvd2_3_r2)

17

In [ ]:
a,b,c = combnfeats_MLR_evaby_crossval_metric(List5_cvd2_3_r2,df_x,df_y,100,model,'r2',0.54)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x99', 'x23', 'x183') with r2 = 0.5494174518807331
Significant!
For features ('x32', 'x23', 'x98', 'x183') with r2 = 0.5414090465260233
Significant!
For features ('x32', 'x23', 'x102', 'x83') with r2 = 0.548351762862512
Significant!
For features ('x32', 'x23', 'x158', 'x140') with r2 = 0.5464205662848712
Significant!
For features ('x32', 'x23', 'x102', 'x79') with r2 = 0.5405632358987134


In [ ]:
a,b,c = combnfeats_MLR_evaby_crossval_metric(List5_cvd2_3_r2,df_x,df_y,500,model,'r2',0.54)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x99', 'x23', 'x183') with r2 = 0.5537395954267417
Significant!
For features ('x32', 'x23', 'x98', 'x183') with r2 = 0.5456166477628911
Significant!
For features ('x32', 'x23', 'x102', 'x83') with r2 = 0.5427734691437951
Significant!
For features ('x32', 'x23', 'x158', 'x140') with r2 = 0.5430511285578842


NMAE

In [ ]:
List_cvd2_3_nmae,List_cvd2_2_nmae,List_cvd2_0_nmae = combnfeats_MLR_evaby_crossval_metric(extended_combinations,df_x,df_y,5,model,'neg_mean_absolute_error',-14.5)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x84') with neg_mean_absolute_error = -13.379371284316386
Significant!
For features ('x32', 'x23', 'x158', 'x193') with neg_mean_absolute_error = -14.070760504124461
Significant!
For features ('x32', 'x23', 'x102', 'x33') with neg_mean_absolute_error = -14.128740250778085
Significant!
For features ('x32', 'x23', 'x102', 'x78') with neg_mean_absolute_error = -14.152271906784671
Significant!
For features ('x32', 'x23', 'x102', 'x82') with neg_mean_absolute_error = -14.09242283350236
Significant!
For features ('x32', 'x99', 'x23', 'x99') with neg_mean_absolute_error = -14.190133608840819
Significant!
For features ('x32', 'x23', 'x158', 'x204') with neg_mean_absolute_error = -14.345531276512085
Significant!
For features ('x32', 'x99', 'x23', 'x157') with neg_mean_absolute_error = -14.341673631064452
Significant!
For features ('x32', 'x23', 'x102', 'x127') with neg_mean_

In [ ]:
List2_cvd2_3_nmae,List2_cvd2_2_nmae,List2_cvd2_0_nmae = combnfeats_MLR_evaby_crossval_metric(List_cvd2_3_nmae,df_x,df_y,5,model,'neg_mean_absolute_error',-13.5)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x84') with neg_mean_absolute_error = -13.379371284316386
Significant!
For features ('x32', 'x23', 'x158', 'x106') with neg_mean_absolute_error = -13.451728832623525
Significant!
For features ('x32', 'x23', 'x102', 'x168') with neg_mean_absolute_error = -13.113385558237905
Significant!
For features ('x23', 'x175', 'x102', 'x32') with neg_mean_absolute_error = -12.831497295221672
Significant!
For features ('x23', 'x175', 'x102', 'x6') with neg_mean_absolute_error = -13.330757171812618
Significant!
For features ('x32', 'x23', 'x158', 'x165') with neg_mean_absolute_error = -13.483611309474094
Significant!
For features ('x32', 'x23', 'x158', 'x83') with neg_mean_absolute_error = -13.07242318824766
Significant!
For features ('x32', 'x23', 'x102', 'x135') with neg_mean_absolute_error = -13.325960836489122
Significant!
For features ('x32', 'x23', 'x102', 'x116') with neg_m

In [ ]:
len(List2_cvd2_3_nmae)

63

In [ ]:
List3_cvd2_3_nmae,List3_cvd2_2_nmae,List3_cvd2_0_nmae = combnfeats_MLR_evaby_crossval_metric(List2_cvd2_3_nmae,df_x,df_y,10,model,'neg_mean_absolute_error',-12.7)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x23', 'x175', 'x102', 'x184') with neg_mean_absolute_error = -12.633891708363455
Significant!
For features ('x32', 'x99', 'x23', 'x183') with neg_mean_absolute_error = -12.460506996390015
Significant!
For features ('x32', 'x23', 'x98', 'x183') with neg_mean_absolute_error = -12.408404716884256
Significant!
For features ('x32', 'x23', 'x102', 'x83') with neg_mean_absolute_error = -12.354069204155985
Significant!
For features ('x32', 'x23', 'x102', 'x77') with neg_mean_absolute_error = -12.4839323128573


In [ ]:
a,b,c = combnfeats_MLR_evaby_crossval_metric(List3_cvd2_3_nmae,df_x,df_y,500,model,'neg_mean_absolute_error',-12.7)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x23', 'x175', 'x102', 'x184') with neg_mean_absolute_error = -12.651508179238114
Significant!
For features ('x32', 'x99', 'x23', 'x183') with neg_mean_absolute_error = -12.541894964324502
Significant!
For features ('x32', 'x23', 'x98', 'x183') with neg_mean_absolute_error = -12.464343450018065
Significant!
For features ('x32', 'x23', 'x102', 'x83') with neg_mean_absolute_error = -12.3706581211348
Significant!
For features ('x32', 'x23', 'x102', 'x77') with neg_mean_absolute_error = -12.519338547233723


Explained variance

In [ ]:
List_2cvd_3_expv,List_2cvd_2_expv,List_2cvd_0_expv = combnfeats_MLR_evaby_crossval_metric(extended_combinations,df_x,df_y,5,model,'explained_variance',0.54)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x206') with explained_variance = 0.5573444041660034
Significant!
For features ('x23', 'x33', 'x98', 'x177') with explained_variance = 0.6183549098333536
Significant!
For features ('x32', 'x23', 'x102', 'x92') with explained_variance = 0.5972823179821395
Significant!
For features ('x23', 'x33', 'x98', 'x206') with explained_variance = 0.5635455151844265
Significant!
For features ('x32', 'x23', 'x159', 'x44') with explained_variance = 0.5678092745054651
Significant!
For features ('x32', 'x99', 'x23', 'x143') with explained_variance = 0.5670339251967664
Significant!
For features ('x32', 'x99', 'x23', 'x177') with explained_variance = 0.6176222934393351
Significant!
For features ('x32', 'x23', 'x102', 'x54') with explained_variance = 0.569868575166501
Significant!
For features ('x32', 'x23', 'x98', 'x125') with explained_variance = 0.5601371261856583
Significant!
For f

In [ ]:
len(List_2cvd_3_expv)

1721

In [ ]:
List2_2cvd_3_expv,List2_2cvd_2_expv,List2_2cvd_0_expv = combnfeats_MLR_evaby_crossval_metric(List_2cvd_3_expv,df_x,df_y,10,model,'explained_variance',0.59)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x23', 'x33', 'x98', 'x177') with explained_variance = 0.6042711975973288
Significant!
For features ('x32', 'x99', 'x23', 'x177') with explained_variance = 0.6354481461221345
Significant!
For features ('x23', 'x175', 'x102', 'x33') with explained_variance = 0.6175340387415906
Significant!
For features ('x32', 'x99', 'x23', 'x79') with explained_variance = 0.6239423888750253
Significant!
For features ('x32', 'x23', 'x158', 'x79') with explained_variance = 0.6209283268190903
Significant!
For features ('x32', 'x23', 'x158', 'x191') with explained_variance = 0.5906858248783987
Significant!
For features ('x23', 'x33', 'x98', 'x183') with explained_variance = 0.5925970597145175
Significant!
For features ('x32', 'x23', 'x158', 'x141') with explained_variance = 0.623870262174702
Significant!
For features ('x32', 'x23', 'x102', 'x176') with explained_variance = 0.6047411664552985
Significant!
Fo

In [ ]:
len(List2_2cvd_3_expv)

116

In [ ]:
List3_2cvd_3_expv,List3_2cvd_2_expv,List3_2cvd_0_expv = combnfeats_MLR_evaby_crossval_metric(List_2cvd_3_expv,df_x,df_y,10,model,'explained_variance',0.62)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x99', 'x23', 'x177') with explained_variance = 0.6354481461221345
Significant!
For features ('x32', 'x99', 'x23', 'x79') with explained_variance = 0.6239423888750253
Significant!
For features ('x32', 'x23', 'x158', 'x79') with explained_variance = 0.6209283268190903
Significant!
For features ('x32', 'x23', 'x158', 'x141') with explained_variance = 0.623870262174702
Significant!
For features ('x32', 'x23', 'x98', 'x83') with explained_variance = 0.6400842220749384
Significant!
For features ('x32', 'x23', 'x158', 'x83') with explained_variance = 0.6203901132999756
Significant!
For features ('x23', 'x33', 'x159', 'x25') with explained_variance = 0.6336086971192367
Significant!
For features ('x32', 'x23', 'x160', 'x140') with explained_variance = 0.6448393351178361
Significant!
For features ('x32', 'x23', 'x158', 'x140') with explained_variance = 0.6558341855233072
Significant!
For 

In [ ]:
len(List3_2cvd_3_expv)

32

In [ ]:
a,b,c = combnfeats_MLR_evaby_crossval_metric(List3_2cvd_3_expv,df_x,df_y,20,model,'explained_variance',0.63)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x83') with explained_variance = 0.630634143777477
Significant!
For features ('x32', 'x23', 'x158', 'x140') with explained_variance = 0.6334751446350394
Significant!
For features ('x32', 'x23', 'x102', 'x83') with explained_variance = 0.6355092502350512
Significant!
For features ('x32', 'x23', 'x102', 'x79') with explained_variance = 0.6302135419853713
Significant!
For features ('x32', 'x99', 'x23', 'x183') with explained_variance = 0.6610315899098994
Significant!
For features ('x32', 'x23', 'x98', 'x183') with explained_variance = 0.6472320924934067


In [ ]:
d,e,f = combnfeats_MLR_evaby_crossval_metric(a,df_x,df_y,500,model,'explained_variance',0.6)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x83') with explained_variance = 0.6195128281318266
Significant!
For features ('x32', 'x23', 'x158', 'x140') with explained_variance = 0.628873881813629
Significant!
For features ('x32', 'x23', 'x102', 'x83') with explained_variance = 0.6302737267546796
Significant!
For features ('x32', 'x23', 'x102', 'x79') with explained_variance = 0.6282862488087938
Significant!
For features ('x32', 'x99', 'x23', 'x183') with explained_variance = 0.6394537011555423
Significant!
For features ('x32', 'x23', 'x98', 'x183') with explained_variance = 0.6322379398341775


Negative RMSE

In [ ]:
List_2cvd_3_nrmse,List_2cvd_2_nrmse,List_2cvd_0_nrmse = combnfeats_MLR_evaby_crossval_metric(extended_combinations,df_x,df_y,5,model,'neg_root_mean_squared_error',-18)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x206') with neg_root_mean_squared_error = -17.561796821674058
Significant!
For features ('x23', 'x33', 'x98', 'x177') with neg_root_mean_squared_error = -17.118128123120353
Significant!
For features ('x32', 'x23', 'x102', 'x92') with neg_root_mean_squared_error = -17.18862617984425
Significant!
For features ('x32', 'x23', 'x159', 'x44') with neg_root_mean_squared_error = -17.787498594119505
Significant!
For features ('x32', 'x99', 'x23', 'x143') with neg_root_mean_squared_error = -17.470408660908053
Significant!
For features ('x32', 'x99', 'x23', 'x177') with neg_root_mean_squared_error = -16.56469857136622
Significant!
For features ('x32', 'x23', 'x102', 'x54') with neg_root_mean_squared_error = -17.629623515591707
Significant!
For features ('x32', 'x23', 'x98', 'x125') with neg_root_mean_squared_error = -17.57070104011064
Significant!
For features ('x23', 'x175',

In [ ]:
len(List_2cvd_3_nrmse)

1123

In [ ]:
List2_2cvd_3_nrmse,List2_2cvd_2_nrmse,List2_2cvd_0_nrmse = combnfeats_MLR_evaby_crossval_metric(List_2cvd_3_nrmse,df_x,df_y,5,model,'neg_root_mean_squared_error',-16.5)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x23', 'x175', 'x102', 'x33') with neg_root_mean_squared_error = -16.216443866830964
Significant!
For features ('x32', 'x99', 'x23', 'x79') with neg_root_mean_squared_error = -16.40803344444949
Significant!
For features ('x32', 'x23', 'x158', 'x79') with neg_root_mean_squared_error = -16.11444914916543
Significant!
For features ('x32', 'x23', 'x158', 'x133') with neg_root_mean_squared_error = -16.353068071944563
Significant!
For features ('x32', 'x23', 'x158', 'x117') with neg_root_mean_squared_error = -16.437659660034875
Significant!
For features ('x32', 'x23', 'x98', 'x83') with neg_root_mean_squared_error = -15.872331909481256
Significant!
For features ('x32', 'x23', 'x158', 'x83') with neg_root_mean_squared_error = -15.865247106204993
Significant!
For features ('x32', 'x23', 'x160', 'x79') with neg_root_mean_squared_error = -16.445405070832628
Significant!
For features ('x23', 'x33'

In [ ]:
len(List2_2cvd_3_nrmse)

52

In [ ]:
List3_2cvd_3_nrmse,b,c = combnfeats_MLR_evaby_crossval_metric(List2_2cvd_3_nrmse,df_x,df_y,10,model,'neg_root_mean_squared_error',-16.1)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x83') with neg_root_mean_squared_error = -16.021281542492208
Significant!
For features ('x32', 'x23', 'x158', 'x140') with neg_root_mean_squared_error = -15.897014237183555
Significant!
For features ('x32', 'x23', 'x102', 'x83') with neg_root_mean_squared_error = -15.990109854300716
Significant!
For features ('x32', 'x23', 'x159', 'x183') with neg_root_mean_squared_error = -16.065538284191682
Significant!
For features ('x32', 'x23', 'x158', 'x209') with neg_root_mean_squared_error = -15.961808239754742
Significant!
For features ('x32', 'x23', 'x158', 'x215') with neg_root_mean_squared_error = -15.978428616075343
Significant!
For features ('x32', 'x99', 'x23', 'x183') with neg_root_mean_squared_error = -15.329628214472516
Significant!
For features ('x32', 'x23', 'x98', 'x183') with neg_root_mean_squared_error = -15.466242695925548


In [ ]:
a,b,c = combnfeats_MLR_evaby_crossval_metric(List3_2cvd_3_nrmse,df_x,df_y,500,model,'neg_root_mean_squared_error',-18)

----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x98', 'x83') with neg_root_mean_squared_error = -15.948198321689258
Significant!
For features ('x32', 'x23', 'x158', 'x140') with neg_root_mean_squared_error = -15.811928631142655
Significant!
For features ('x32', 'x23', 'x102', 'x83') with neg_root_mean_squared_error = -15.858839231532627
Significant!
For features ('x32', 'x23', 'x159', 'x183') with neg_root_mean_squared_error = -15.816745786810042
Significant!
For features ('x32', 'x23', 'x158', 'x209') with neg_root_mean_squared_error = -15.941578936975493
Significant!
For features ('x32', 'x23', 'x158', 'x215') with neg_root_mean_squared_error = -15.906123360862258
Significant!
For features ('x32', 'x99', 'x23', 'x183') with neg_root_mean_squared_error = -15.26362317510753
Significant!
For features ('x32', 'x23', 'x98', 'x183') with neg_root_mean_squared_error = -15.411573553401889


max error

In [9]:
List2_2cvd_3_me,List2_2cvd_2_me,List2_2cvd_0_me = combnfeats_MLR_evaby_crossval_metric(extended_combinations,df_x,df_y,5,model,'max_error',-31)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x23', 'x33', 'x159', 'x53') with max_error = -30.94780885268535
Significant!
For features ('x32', 'x23', 'x158', 'x158') with max_error = -30.82107893785972
Significant!
For features ('x23', 'x33', 'x98', 'x109') with max_error = -30.710933063473533
Significant!
For features ('x32', 'x23', 'x158', 'x116') with max_error = -30.066850870563304
Significant!
For features ('x32', 'x23', 'x98', 'x83') with max_error = -30.658826249291952
Significant!
For features ('x32', 'x99', 'x23', 'x168') with max_error = -30.858957121009972
Significant!
For features ('x23', 'x33', 'x158', 'x25') with max_error = -29.19920368434642
Significant!
For features ('x32', 'x23', 'x158', 'x150') with max_error = -30.063636870131507
Significant!
For features ('x32', 'x23', 'x160', 'x114') with max_error = -30.913239167857803
Significant!
For features ('x99', 'x23', 'x33', 'x175') with max_error = -30.936430704493

In [10]:
len(List2_2cvd_3_me)

151

In [11]:
List3_2cvd_3_me,b,c = combnfeats_MLR_evaby_crossval_metric(List2_2cvd_3_me,df_x,df_y,10,model,'max_error',-30)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x23', 'x33', 'x98', 'x109') with max_error = -29.79114401371577
Significant!
For features ('x32', 'x23', 'x158', 'x116') with max_error = -29.493223670689396
Significant!
For features ('x23', 'x33', 'x158', 'x25') with max_error = -29.90319257458386
Significant!
For features ('x32', 'x23', 'x158', 'x113') with max_error = -29.658267220148844
Significant!
For features ('x32', 'x23', 'x158', 'x165') with max_error = -28.21865816273101
Significant!
For features ('x32', 'x23', 'x160', 'x105') with max_error = -29.403741543127516
Significant!
For features ('x32', 'x23', 'x160', 'x209') with max_error = -28.15180725852195
Significant!
For features ('x32', 'x23', 'x158', 'x140') with max_error = -29.547672035694752
Significant!
For features ('x32', 'x23', 'x158', 'x136') with max_error = -29.765716842833548
Significant!
For features ('x32', 'x23', 'x160', 'x215') with max_error = -28.54840136

In [12]:
len(List3_2cvd_3_me)

44

In [13]:
List4_2cvd_3_me,b,c = combnfeats_MLR_evaby_crossval_metric(List3_2cvd_3_me,df_x,df_y,20,model,'max_error',-29)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x158', 'x165') with max_error = -28.603417123903466
Significant!
For features ('x32', 'x23', 'x160', 'x209') with max_error = -28.13246886475947
Significant!
For features ('x32', 'x23', 'x160', 'x215') with max_error = -28.891945893493848
Significant!
For features ('x32', 'x23', 'x158', 'x163') with max_error = -27.930868429870337
Significant!
For features ('x32', 'x23', 'x158', 'x215') with max_error = -28.73248496747511
Significant!
For features ('x23', 'x33', 'x159', 'x25') with max_error = -28.489178174321733
Significant!
For features ('x32', 'x23', 'x160', 'x165') with max_error = -28.334511165821517
Significant!
For features ('x32', 'x23', 'x158', 'x209') with max_error = -28.06103306740787
Significant!
For features ('x32', 'x23', 'x160', 'x163') with max_error = -27.97371160135075
Significant!
For features ('x23', 'x33', 'x158', 'x109') with max_error = -28.6043958

In [14]:
a,b,c = combnfeats_MLR_evaby_crossval_metric(List4_2cvd_3_me,df_x,df_y,500,model,'max_error',-29)


----------Linear regression of n features evaby crossval starts----------
Significant!
For features ('x32', 'x23', 'x158', 'x165') with max_error = -27.933587397600462
Significant!
For features ('x32', 'x23', 'x160', 'x209') with max_error = -28.053624597910908
Significant!
For features ('x32', 'x23', 'x160', 'x215') with max_error = -28.463495321255532
Significant!
For features ('x32', 'x23', 'x158', 'x163') with max_error = -27.380171005699907
Significant!
For features ('x32', 'x23', 'x158', 'x215') with max_error = -28.192910663922238
Significant!
For features ('x23', 'x33', 'x159', 'x25') with max_error = -27.999930750224785
Significant!
For features ('x32', 'x23', 'x160', 'x165') with max_error = -27.85489414464943
Significant!
For features ('x32', 'x23', 'x158', 'x209') with max_error = -27.671359599400226
Significant!
For features ('x32', 'x23', 'x160', 'x163') with max_error = -27.6841707943629
Significant!
For features ('x23', 'x33', 'x158', 'x109') with max_error = -28.358114